In [1]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import json
import re
from pprint import pprint

In [2]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
# Launch browser
browser = Browser ('chrome')
url = 'https://en.wikipedia.org/wiki/List_of_United_States_Air_Force_installations'
browser.visit(url)

The chromedriver version (125.0.6422.78) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (126.0.6478.116); currently, chromedriver 126.0.6478.126 is recommended for chrome 126.*, so it is advised to delete the driver in PATH and retry


In [4]:
# Create Beautiful Soup
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find('table', class_='wikitable sortable jquery-tablesorter')
table

<table class="wikitable sortable jquery-tablesorter">
<caption>Active Duty US Air Force installations and locations within the contiguous United States
</caption>
<thead><tr>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Name
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Location
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">State or area
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Coordinates
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Commanding organization
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Wing or unit emblem
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">Host wing or primary unit
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="S

In [5]:
# Standardize header names
def standardize_header_name(header):
    return header.replace(' ', '_').replace('-', '_').lower()

In [6]:
# Extract the header row
headers = [standardize_header_name(th.text.strip()) for th in table.find('tr').find_all('th')]
headers

['name',
 'location',
 'state_or_area',
 'coordinates',
 'commanding_organization',
 'wing_or_unit_emblem',
 'host_wing_or_primary_unit',
 'primary_missions_and_units']

In [7]:
# Create an empty list
active_duty_us_bases = []
rows = table.find_all('tr')[1:] # skip the first row as it contains headers

# Loop through the table rows to scrape data
for row in rows:
    row_data = [td.get_text(" ", strip=True).replace('\n', ' ') for td in row.find_all('td')]
    active_duty_us_bases.append(row_data)

pprint(active_duty_us_bases)

[['Altus Air Force Base',
  'Altus',
  'Oklahoma',
  '34°39′59″N 099°16′05″W \ufeff / \ufeff 34.66639°N 99.26806°W \ufeff / '
  '34.66639; -99.26806 \ufeff ( Altus AFB )',
  'Air Education and Training Command',
  '',
  '97th Air Mobility Wing',
  'The 97th Air Mobility Wing trains crews to operate the C-17A Globemaster '
  'III , KC-135R Stratotanker and KC-46A Pegasus . [4]'],
 ['Joint Base Anacostia-Bolling',
  'Southwest',
  'Washington, D.C.',
  '38°50′34″N 077°00′58″W \ufeff / \ufeff 38.84278°N 77.01611°W \ufeff / '
  '38.84278; -77.01611 \ufeff ( JB Anacostia-Bolling )',
  'Air Force District of Washington',
  '',
  '11th Wing',
  'USAF operated joint base . The 11th Wing provides base administration and '
  'support to tenants including the USAF Honor Guard and USAF Band , which '
  'support ceremonial operations in the National Capital Region . [5]'],
 ['Joint Base Andrews-Naval Air Facility Washington',
  'Camp Springs',
  'Maryland',
  '38°48′39″N 076°52′01″W \ufeff / \ufeff

In [8]:
# Creating the DataFrame
active_duty_df = pd.DataFrame(active_duty_us_bases, columns=headers)
active_duty_df

,name,location,state_or_area,coordinates,commanding_organization,wing_or_unit_emblem,host_wing_or_primary_unit,primary_missions_and_units
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W ﻿ / ﻿ 34.66639°N 99.268...,Air Education and Training Command,,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W ﻿ / ﻿ 38.84278°N 77.016...,Air Force District of Washington,,11th Wing,USAF operated joint base . The 11th Wing provi...
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W ﻿ / ﻿ 38.81083°N 76.866...,Air Force District of Washington,,316th Wing,USAF operated joint base . The 316th Wing prov...
3,Arnold Air Force Base,Tullahoma,Tennessee,35°23′33″N 086°05′09″W ﻿ / ﻿ 35.39250°N 86.085...,Air Force Materiel Command,,Arnold Engineering Development Complex,"Non-flying installation, part of the Air Force..."
4,Barksdale Air Force Base,Bossier City,Louisiana,32°30′07″N 093°39′46″W ﻿ / ﻿ 32.50194°N 93.662...,Air Force Global Strike Command,,2nd Bomb Wing,The 2nd Bomb Wing and 307th Bomb Wing operate ...
...,...,...,...,...,...,...,...,...
61,United States Air Force Academy,Colorado Springs,Colorado,38°59′25″N 104°51′30″W ﻿ / ﻿ 38.99028°N 104.85...,Chief of Staff of the Air Force,,10th Air Base Wing,Military academy for the US Air Force and US S...
62,Vance Air Force Base,Enid,Oklahoma,36°20′22″N 097°55′02″W ﻿ / ﻿ 36.33944°N 97.917...,Air Education and Training Command,,71st Flying Training Wing,71st Flying Training Wing and 5th Flying Train...
63,Vandenberg Space Force Base,Lompoc,California,34°43′57″N 120°34′05″W ﻿ / ﻿ 34.73250°N 120.56...,Various,,N/A,US Space Force installation hosting the 532nd ...
64,Whiteman Air Force Base,Knob Noster,Missouri,38°43′49″N 093°32′53″W ﻿ / ﻿ 38.73028°N 93.548...,Air Force Global Strike Command,,509th Bomb Wing,The 509th Bomb Wing and the Missouri Air Natio...


In [9]:
# Split the 'Coordinates' column into two new columns
split_df = active_duty_df['coordinates'].str.split(' / ', expand=True)

# Ensure there are always two columns
if split_df.shape[1] == 1:
    
    # Add a second column with NaN if only one column resulted from the split
    split_df[1] = pd.NA

# Assign split columns to the original DataFrame
active_duty_df['dms_coordinates'] = split_df[0]
active_duty_df['decimal_coordinates'] = split_df[1]

# Print the DataFrame to verify the output
print(active_duty_df[['dms_coordinates', 'decimal_coordinates']])

             dms_coordinates         decimal_coordinates
0   34°39′59″N 099°16′05″W ﻿   ﻿ 34.66639°N 99.26806°W ﻿
1   38°50′34″N 077°00′58″W ﻿   ﻿ 38.84278°N 77.01611°W ﻿
2   38°48′39″N 076°52′01″W ﻿   ﻿ 38.81083°N 76.86694°W ﻿
3   35°23′33″N 086°05′09″W ﻿   ﻿ 35.39250°N 86.08583°W ﻿
4   32°30′07″N 093°39′46″W ﻿   ﻿ 32.50194°N 93.66278°W ﻿
..                       ...                         ...
61  38°59′25″N 104°51′30″W ﻿  ﻿ 38.99028°N 104.85833°W ﻿
62  36°20′22″N 097°55′02″W ﻿   ﻿ 36.33944°N 97.91722°W ﻿
63  34°43′57″N 120°34′05″W ﻿  ﻿ 34.73250°N 120.56806°W ﻿
64  38°43′49″N 093°32′53″W ﻿   ﻿ 38.73028°N 93.54806°W ﻿
65  39°49′23″N 084°02′58″W ﻿   ﻿ 39.82306°N 84.04944°W ﻿

[66 rows x 2 columns]


In [10]:
active_duty_df

,name,location,state_or_area,coordinates,commanding_organization,wing_or_unit_emblem,host_wing_or_primary_unit,primary_missions_and_units,dms_coordinates,decimal_coordinates
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W ﻿ / ﻿ 34.66639°N 99.268...,Air Education and Training Command,,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34°39′59″N 099°16′05″W ﻿,﻿ 34.66639°N 99.26806°W ﻿
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W ﻿ / ﻿ 38.84278°N 77.016...,Air Force District of Washington,,11th Wing,USAF operated joint base . The 11th Wing provi...,38°50′34″N 077°00′58″W ﻿,﻿ 38.84278°N 77.01611°W ﻿
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W ﻿ / ﻿ 38.81083°N 76.866...,Air Force District of Washington,,316th Wing,USAF operated joint base . The 316th Wing prov...,38°48′39″N 076°52′01″W ﻿,﻿ 38.81083°N 76.86694°W ﻿
3,Arnold Air Force Base,Tullahoma,Tennessee,35°23′33″N 086°05′09″W ﻿ / ﻿ 35.39250°N 86.085...,Air Force Materiel Command,,Arnold Engineering Development Complex,"Non-flying installation, part of the Air Force...",35°23′33″N 086°05′09″W ﻿,﻿ 35.39250°N 86.08583°W ﻿
4,Barksdale Air Force Base,Bossier City,Louisiana,32°30′07″N 093°39′46″W ﻿ / ﻿ 32.50194°N 93.662...,Air Force Global Strike Command,,2nd Bomb Wing,The 2nd Bomb Wing and 307th Bomb Wing operate ...,32°30′07″N 093°39′46″W ﻿,﻿ 32.50194°N 93.66278°W ﻿
...,...,...,...,...,...,...,...,...,...,...
61,United States Air Force Academy,Colorado Springs,Colorado,38°59′25″N 104°51′30″W ﻿ / ﻿ 38.99028°N 104.85...,Chief of Staff of the Air Force,,10th Air Base Wing,Military academy for the US Air Force and US S...,38°59′25″N 104°51′30″W ﻿,﻿ 38.99028°N 104.85833°W ﻿
62,Vance Air Force Base,Enid,Oklahoma,36°20′22″N 097°55′02″W ﻿ / ﻿ 36.33944°N 97.917...,Air Education and Training Command,,71st Flying Training Wing,71st Flying Training Wing and 5th Flying Train...,36°20′22″N 097°55′02″W ﻿,﻿ 36.33944°N 97.91722°W ﻿
63,Vandenberg Space Force Base,Lompoc,California,34°43′57″N 120°34′05″W ﻿ / ﻿ 34.73250°N 120.56...,Various,,N/A,US Space Force installation hosting the 532nd ...,34°43′57″N 120°34′05″W ﻿,﻿ 34.73250°N 120.56806°W ﻿
64,Whiteman Air Force Base,Knob Noster,Missouri,38°43′49″N 093°32′53″W ﻿ / ﻿ 38.73028°N 93.548...,Air Force Global Strike Command,,509th Bomb Wing,The 509th Bomb Wing and the Missouri Air Natio...,38°43′49″N 093°32′53″W ﻿,﻿ 38.73028°N 93.54806°W ﻿


In [11]:
# Clean and convert the Decimal_Coordinates column by extracting just the numeral
def clean_and_format_coords(coord):
    
    if pd.isna(coord):
        return None  # Handle missing values
    
    # Extract numbers and potential N, S, E, W characters
    parts = re.findall(r'([+-]?\d+\.\d+)([NSEW]?)', coord)
    
    if not parts:
        return None
    
    # Prepare latitude and longitude, considering the direction
    cleaned_parts = []
    
    for value, direction in parts:
        num = float(value)

        # South or West should be negative
        if direction == 'S' or direction == 'W':
            num = -num
        cleaned_parts.append(num)
    
    if len(cleaned_parts) == 2:
        # Ensure longitude comes first for GeoJSON
        return (cleaned_parts[1], cleaned_parts[0])
    return None

# Apply this cleaning and formatting to the DataFrame
active_duty_df['geojson_coordinates'] = active_duty_df['decimal_coordinates'].apply(clean_and_format_coords)
active_duty_df

,name,location,state_or_area,coordinates,commanding_organization,wing_or_unit_emblem,host_wing_or_primary_unit,primary_missions_and_units,dms_coordinates,decimal_coordinates,geojson_coordinates
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W ﻿ / ﻿ 34.66639°N 99.268...,Air Education and Training Command,,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34°39′59″N 099°16′05″W ﻿,﻿ 34.66639°N 99.26806°W ﻿,"(99.26806, 34.66639)"
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W ﻿ / ﻿ 38.84278°N 77.016...,Air Force District of Washington,,11th Wing,USAF operated joint base . The 11th Wing provi...,38°50′34″N 077°00′58″W ﻿,﻿ 38.84278°N 77.01611°W ﻿,"(77.01611, 38.84278)"
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W ﻿ / ﻿ 38.81083°N 76.866...,Air Force District of Washington,,316th Wing,USAF operated joint base . The 316th Wing prov...,38°48′39″N 076°52′01″W ﻿,﻿ 38.81083°N 76.86694°W ﻿,"(76.86694, 38.81083)"
3,Arnold Air Force Base,Tullahoma,Tennessee,35°23′33″N 086°05′09″W ﻿ / ﻿ 35.39250°N 86.085...,Air Force Materiel Command,,Arnold Engineering Development Complex,"Non-flying installation, part of the Air Force...",35°23′33″N 086°05′09″W ﻿,﻿ 35.39250°N 86.08583°W ﻿,"(86.08583, 35.3925)"
4,Barksdale Air Force Base,Bossier City,Louisiana,32°30′07″N 093°39′46″W ﻿ / ﻿ 32.50194°N 93.662...,Air Force Global Strike Command,,2nd Bomb Wing,The 2nd Bomb Wing and 307th Bomb Wing operate ...,32°30′07″N 093°39′46″W ﻿,﻿ 32.50194°N 93.66278°W ﻿,"(93.66278, 32.50194)"
...,...,...,...,...,...,...,...,...,...,...,...
61,United States Air Force Academy,Colorado Springs,Colorado,38°59′25″N 104°51′30″W ﻿ / ﻿ 38.99028°N 104.85...,Chief of Staff of the Air Force,,10th Air Base Wing,Military academy for the US Air Force and US S...,38°59′25″N 104°51′30″W ﻿,﻿ 38.99028°N 104.85833°W ﻿,"(104.85833, 38.99028)"
62,Vance Air Force Base,Enid,Oklahoma,36°20′22″N 097°55′02″W ﻿ / ﻿ 36.33944°N 97.917...,Air Education and Training Command,,71st Flying Training Wing,71st Flying Training Wing and 5th Flying Train...,36°20′22″N 097°55′02″W ﻿,﻿ 36.33944°N 97.91722°W ﻿,"(97.91722, 36.33944)"
63,Vandenberg Space Force Base,Lompoc,California,34°43′57″N 120°34′05″W ﻿ / ﻿ 34.73250°N 120.56...,Various,,N/A,US Space Force installation hosting the 532nd ...,34°43′57″N 120°34′05″W ﻿,﻿ 34.73250°N 120.56806°W ﻿,"(120.56806, 34.7325)"
64,Whiteman Air Force Base,Knob Noster,Missouri,38°43′49″N 093°32′53″W ﻿ / ﻿ 38.73028°N 93.548...,Air Force Global Strike Command,,509th Bomb Wing,The 509th Bomb Wing and the Missouri Air Natio...,38°43′49″N 093°32′53″W ﻿,﻿ 38.73028°N 93.54806°W ﻿,"(93.54806, 38.73028)"


In [12]:
# Split the 'GeoJSON_Coordinates' into 'Longitude' and 'Latitude'
active_duty_df['longitude'], active_duty_df['latitude'] = zip(*active_duty_df['geojson_coordinates'])

# Print the updated DataFrame to check the new columns
print(active_duty_df[['longitude', 'latitude']])

    longitude  latitude
0    99.26806  34.66639
1    77.01611  38.84278
2    76.86694  38.81083
3    86.08583  35.39250
4    93.66278  32.50194
..        ...       ...
61  104.85833  38.99028
62   97.91722  36.33944
63  120.56806  34.73250
64   93.54806  38.73028
65   84.04944  39.82306

[66 rows x 2 columns]


In [13]:
active_duty_df

,name,location,state_or_area,coordinates,commanding_organization,wing_or_unit_emblem,host_wing_or_primary_unit,primary_missions_and_units,dms_coordinates,decimal_coordinates,geojson_coordinates,longitude,latitude
0,Altus Air Force Base,Altus,Oklahoma,34°39′59″N 099°16′05″W ﻿ / ﻿ 34.66639°N 99.268...,Air Education and Training Command,,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,34°39′59″N 099°16′05″W ﻿,﻿ 34.66639°N 99.26806°W ﻿,"(99.26806, 34.66639)",99.26806,34.66639
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",38°50′34″N 077°00′58″W ﻿ / ﻿ 38.84278°N 77.016...,Air Force District of Washington,,11th Wing,USAF operated joint base . The 11th Wing provi...,38°50′34″N 077°00′58″W ﻿,﻿ 38.84278°N 77.01611°W ﻿,"(77.01611, 38.84278)",77.01611,38.84278
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,38°48′39″N 076°52′01″W ﻿ / ﻿ 38.81083°N 76.866...,Air Force District of Washington,,316th Wing,USAF operated joint base . The 316th Wing prov...,38°48′39″N 076°52′01″W ﻿,﻿ 38.81083°N 76.86694°W ﻿,"(76.86694, 38.81083)",76.86694,38.81083
3,Arnold Air Force Base,Tullahoma,Tennessee,35°23′33″N 086°05′09″W ﻿ / ﻿ 35.39250°N 86.085...,Air Force Materiel Command,,Arnold Engineering Development Complex,"Non-flying installation, part of the Air Force...",35°23′33″N 086°05′09″W ﻿,﻿ 35.39250°N 86.08583°W ﻿,"(86.08583, 35.3925)",86.08583,35.39250
4,Barksdale Air Force Base,Bossier City,Louisiana,32°30′07″N 093°39′46″W ﻿ / ﻿ 32.50194°N 93.662...,Air Force Global Strike Command,,2nd Bomb Wing,The 2nd Bomb Wing and 307th Bomb Wing operate ...,32°30′07″N 093°39′46″W ﻿,﻿ 32.50194°N 93.66278°W ﻿,"(93.66278, 32.50194)",93.66278,32.50194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,United States Air Force Academy,Colorado Springs,Colorado,38°59′25″N 104°51′30″W ﻿ / ﻿ 38.99028°N 104.85...,Chief of Staff of the Air Force,,10th Air Base Wing,Military academy for the US Air Force and US S...,38°59′25″N 104°51′30″W ﻿,﻿ 38.99028°N 104.85833°W ﻿,"(104.85833, 38.99028)",104.85833,38.99028
62,Vance Air Force Base,Enid,Oklahoma,36°20′22″N 097°55′02″W ﻿ / ﻿ 36.33944°N 97.917...,Air Education and Training Command,,71st Flying Training Wing,71st Flying Training Wing and 5th Flying Train...,36°20′22″N 097°55′02″W ﻿,﻿ 36.33944°N 97.91722°W ﻿,"(97.91722, 36.33944)",97.91722,36.33944
63,Vandenberg Space Force Base,Lompoc,California,34°43′57″N 120°34′05″W ﻿ / ﻿ 34.73250°N 120.56...,Various,,N/A,US Space Force installation hosting the 532nd ...,34°43′57″N 120°34′05″W ﻿,﻿ 34.73250°N 120.56806°W ﻿,"(120.56806, 34.7325)",120.56806,34.73250
64,Whiteman Air Force Base,Knob Noster,Missouri,38°43′49″N 093°32′53″W ﻿ / ﻿ 38.73028°N 93.548...,Air Force Global Strike Command,,509th Bomb Wing,The 509th Bomb Wing and the Missouri Air Natio...,38°43′49″N 093°32′53″W ﻿,﻿ 38.73028°N 93.54806°W ﻿,"(93.54806, 38.73028)",93.54806,38.73028


In [14]:
# Drop the columns that are no longer needed
active_duty_df = active_duty_df.drop(columns=[
    'coordinates', 
    'wing_or_unit_emblem', 
    'dms_coordinates', 
    'decimal_coordinates', 
    'geojson_coordinates'
])

active_duty_df

,name,location,state_or_area,commanding_organization,host_wing_or_primary_unit,primary_missions_and_units,longitude,latitude
0,Altus Air Force Base,Altus,Oklahoma,Air Education and Training Command,97th Air Mobility Wing,The 97th Air Mobility Wing trains crews to ope...,99.26806,34.66639
1,Joint Base Anacostia-Bolling,Southwest,"Washington, D.C.",Air Force District of Washington,11th Wing,USAF operated joint base . The 11th Wing provi...,77.01611,38.84278
2,Joint Base Andrews-Naval Air Facility Washington,Camp Springs,Maryland,Air Force District of Washington,316th Wing,USAF operated joint base . The 316th Wing prov...,76.86694,38.81083
3,Arnold Air Force Base,Tullahoma,Tennessee,Air Force Materiel Command,Arnold Engineering Development Complex,"Non-flying installation, part of the Air Force...",86.08583,35.39250
4,Barksdale Air Force Base,Bossier City,Louisiana,Air Force Global Strike Command,2nd Bomb Wing,The 2nd Bomb Wing and 307th Bomb Wing operate ...,93.66278,32.50194
...,...,...,...,...,...,...,...,...
61,United States Air Force Academy,Colorado Springs,Colorado,Chief of Staff of the Air Force,10th Air Base Wing,Military academy for the US Air Force and US S...,104.85833,38.99028
62,Vance Air Force Base,Enid,Oklahoma,Air Education and Training Command,71st Flying Training Wing,71st Flying Training Wing and 5th Flying Train...,97.91722,36.33944
63,Vandenberg Space Force Base,Lompoc,California,Various,N/A,US Space Force installation hosting the 532nd ...,120.56806,34.73250
64,Whiteman Air Force Base,Knob Noster,Missouri,Air Force Global Strike Command,509th Bomb Wing,The 509th Bomb Wing and the Missouri Air Natio...,93.54806,38.73028


In [15]:
# Convert DataFrame to a JSON string
json_str = active_duty_df.to_json(orient='records')

# Parse the JSON string back into a Python list of dictionaries
data = json.loads(json_str)

# Write the JSON file with indentation for better readability
with open('afb_active_duty.json', 'w') as f:
    json.dump(data, f, indent=4)

# Optionally, print the JSON string to the console for verification
print(json.dumps(data, indent=4))

[
    {
        "name": "Altus Air Force Base",
        "location": "Altus",
        "state_or_area": "Oklahoma",
        "commanding_organization": "Air Education and Training Command",
        "host_wing_or_primary_unit": "97th Air Mobility Wing",
        "primary_missions_and_units": "The 97th Air Mobility Wing trains crews to operate the C-17A Globemaster III , KC-135R Stratotanker and KC-46A Pegasus . [4]",
        "longitude": 99.26806,
        "latitude": 34.66639
    },
    {
        "name": "Joint Base Anacostia-Bolling",
        "location": "Southwest",
        "state_or_area": "Washington, D.C.",
        "commanding_organization": "Air Force District of Washington",
        "host_wing_or_primary_unit": "11th Wing",
        "primary_missions_and_units": "USAF operated joint base . The 11th Wing provides base administration and support to tenants including the USAF Honor Guard and USAF Band , which support ceremonial operations in the National Capital Region . [5]",
        "l